# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 3 2022 8:56AM,243335,10,0085921607,ISDIN Corporation,Released
1,Aug 3 2022 8:56AM,243335,10,0085921618,ISDIN Corporation,Released
2,Aug 3 2022 8:56AM,243335,10,0085921620,ISDIN Corporation,Released
3,Aug 3 2022 8:56AM,243335,10,0085921615,ISDIN Corporation,Released
4,Aug 3 2022 8:56AM,243335,10,0085921621,ISDIN Corporation,Released
5,Aug 3 2022 8:56AM,243335,10,0085921619,ISDIN Corporation,Released
6,Aug 3 2022 8:56AM,243335,10,0085921617,ISDIN Corporation,Released
7,Aug 3 2022 8:56AM,243335,10,0085921623,ISDIN Corporation,Released
8,Aug 3 2022 8:56AM,243335,10,0085921616,ISDIN Corporation,Released
9,Aug 3 2022 8:56AM,243335,10,0085921622,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,243328,Released,7
27,243330,Released,3
28,243331,Released,12
29,243334,Released,15
30,243335,Released,84


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243328,NaN,NaN,7.0
243330,NaN,NaN,3.0
243331,NaN,NaN,12.0
243334,NaN,NaN,15.0
243335,NaN,NaN,84.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243262,0.0,0.0,1.0
243270,0.0,0.0,1.0
243274,13.0,7.0,7.0
243283,6.0,3.0,0.0
243294,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243262,0,0,1
243270,0,0,1
243274,13,7,7
243283,6,3,0
243294,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243262,0,0,1
1,243270,0,0,1
2,243274,13,7,7
3,243283,6,3,0
4,243294,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243262,,,1
1,243270,,,1
2,243274,13,7,7
3,243283,6,3,
4,243294,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 3 2022 8:56AM,243335,10,ISDIN Corporation
84,Aug 3 2022 8:42AM,243334,10,ISDIN Corporation
99,Aug 3 2022 8:30AM,243331,10,ISDIN Corporation
111,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC"
112,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation
121,Aug 3 2022 8:30AM,243312,10,SugarBear
122,Aug 3 2022 8:11AM,243330,10,Emerginnova
125,Aug 3 2022 8:04AM,243328,19,"AdvaGen Pharma, Ltd"
132,Aug 3 2022 8:02AM,243327,19,"AdvaGen Pharma, Ltd"
136,Aug 2 2022 5:05PM,243325,18,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 3 2022 8:56AM,243335,10,ISDIN Corporation,,,84
1,Aug 3 2022 8:42AM,243334,10,ISDIN Corporation,,,15
2,Aug 3 2022 8:30AM,243331,10,ISDIN Corporation,,,12
3,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",,,1
4,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,,,9
5,Aug 3 2022 8:30AM,243312,10,SugarBear,,,1
6,Aug 3 2022 8:11AM,243330,10,Emerginnova,,,3
7,Aug 3 2022 8:04AM,243328,19,"AdvaGen Pharma, Ltd",,,7
8,Aug 3 2022 8:02AM,243327,19,"AdvaGen Pharma, Ltd",,,4
9,Aug 2 2022 5:05PM,243325,18,Innova Softgel LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:56AM,243335,10,ISDIN Corporation,84,,
1,Aug 3 2022 8:42AM,243334,10,ISDIN Corporation,15,,
2,Aug 3 2022 8:30AM,243331,10,ISDIN Corporation,12,,
3,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1,,
4,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9,,
5,Aug 3 2022 8:30AM,243312,10,SugarBear,1,,
6,Aug 3 2022 8:11AM,243330,10,Emerginnova,3,,
7,Aug 3 2022 8:04AM,243328,19,"AdvaGen Pharma, Ltd",7,,
8,Aug 3 2022 8:02AM,243327,19,"AdvaGen Pharma, Ltd",4,,
9,Aug 2 2022 5:05PM,243325,18,Innova Softgel LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:56AM,243335,10,ISDIN Corporation,84,,
1,Aug 3 2022 8:42AM,243334,10,ISDIN Corporation,15,,
2,Aug 3 2022 8:30AM,243331,10,ISDIN Corporation,12,,
3,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1,,
4,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:56AM,243335,10,ISDIN Corporation,84.0,NaN,NaN
1,Aug 3 2022 8:42AM,243334,10,ISDIN Corporation,15.0,NaN,NaN
2,Aug 3 2022 8:30AM,243331,10,ISDIN Corporation,12.0,NaN,NaN
3,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 3 2022 8:56AM,243335,10,ISDIN Corporation,84.0,0.0,0.0
1,Aug 3 2022 8:42AM,243334,10,ISDIN Corporation,15.0,0.0,0.0
2,Aug 3 2022 8:30AM,243331,10,ISDIN Corporation,12.0,0.0,0.0
3,Aug 3 2022 8:30AM,243316,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Aug 3 2022 8:30AM,243311,10,ISDIN Corporation,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4136170,143.0,10.0,19.0
12,243324,1.0,0.0,0.0
18,243325,1.0,0.0,0.0
19,729974,26.0,5.0,0.0
21,1459913,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4136170,143.0,10.0,19.0
1,12,243324,1.0,0.0,0.0
2,18,243325,1.0,0.0,0.0
3,19,729974,26.0,5.0,0.0
4,21,1459913,6.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,143.0,10.0,19.0
1,12,1.0,0.0,0.0
2,18,1.0,0.0,0.0
3,19,26.0,5.0,0.0
4,21,6.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,143.0
1,12,Released,1.0
2,18,Released,1.0
3,19,Released,26.0
4,21,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,18,19,21
Status,,,,,
Completed,19.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,0.0,5.0,0.0
Released,143.0,1.0,1.0,26.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,18,19,21
0,Completed,19.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,5.0,0.0
2,Released,143.0,1.0,1.0,26.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,18,19,21
0,Completed,19.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,5.0,0.0
2,Released,143.0,1.0,1.0,26.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()